In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('fake_news.csv')
print(data['title'][0])
data.head(2)

House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0


In [4]:
X = data.drop('label',axis=1)

In [5]:
y = data['label']

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [7]:
data = data.dropna()

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18285 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18285 non-null  int64 
 1   title   18285 non-null  object
 2   author  18285 non-null  object
 3   text    18285 non-null  object
 4   label   18285 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 857.1+ KB


In [9]:
msgs = data.copy()

In [10]:
msgs.reset_index(inplace = True)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [12]:
import regex as re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(msgs)):
    review = re.sub('[^a-zA-Z]',' ',msgs['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [13]:
cv = CountVectorizer(max_features=5000,ngram_range = (1,3))
X = cv.fit_transform(corpus).toarray()

In [14]:
cv.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': 5000,
 'min_df': 1,
 'ngram_range': (1, 3),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [15]:
y = msgs['label']
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB(alpha=0.1)
from sklearn.metrics import accuracy_score
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25,random_state = 0)

In [16]:
prev_score = 0
for alpha in np.arange(0.1,1,0.1):
    mnb1 = MultinomialNB(alpha=alpha)
    mnb1.fit(X_train,y_train)
    pred = mnb1.predict(X_test)
    score = accuracy_score(y_test,pred)
    if score>prev_score:
        mnb = mnb1
    print("Alpha: {},score: {}".format(alpha,score))
    

Alpha: 0.1,score: 0.9041994750656168
Alpha: 0.2,score: 0.9041994750656168
Alpha: 0.30000000000000004,score: 0.9046369203849519
Alpha: 0.4,score: 0.9037620297462817
Alpha: 0.5,score: 0.902230971128609
Alpha: 0.6,score: 0.902230971128609
Alpha: 0.7000000000000001,score: 0.9024496937882764
Alpha: 0.8,score: 0.902230971128609
Alpha: 0.9,score: 0.9017935258092739


In [18]:
# Import necessary libraries
import numpy as np

# Function for inline fake news classification
def classify_fake_news(title):
    # Preprocess the input title
    review = re.sub('[^a-zA-Z]', ' ', title)
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    
    # Vectorize the processed title
    title_vector = cv.transform([review]).toarray()
    
    # Predict the label
    prediction = mnb.predict(title_vector)
    
    # Return the result
    return "Fake News" if prediction[0] == 1 else "Real News"

In [27]:
test_title = input()
result = classify_fake_news(test_title)
print(f"Title: {test_title}\nPrediction: {result}")

 'Jackie Mason: Hollywood Would Love Trump if He Bombed North Korea over Lack of Trans Bathrooms (Exclusive Video) - Breitbart


Title: 'Jackie Mason: Hollywood Would Love Trump if He Bombed North Korea over Lack of Trans Bathrooms (Exclusive Video) - Breitbart
Prediction: Real News


In [28]:
import pickle

   # Save the model
with open('model.pkl', 'wb') as model_file:
    pickle.dump(mnb, model_file)

   # Save the CountVectorizer
with open('cv.pkl', 'wb') as vectorizer_file:
    pickle.dump(cv, vectorizer_file)